# Tools to compare outputs among models
Last modification: 2024.04.09.</br>
Update note: Make things compact

# Plot summary

In [ ]:
from perturb_plots import plot_summary
import matplotlib.pylab as plt

law = 'aging'
ax = plot_summary(law=law)
plt.tight_layout()
plt.savefig('/export/dump/jyun/perturb_stress/%slaw_summary.png'%(law),dpi=300)
plt.show()

# Routined

In [ ]:
from perturb_tools import ROUTINE_PERTURB
import matplotlib.pylab as plt
plt.rcParams['font.size'] = '15'

routine = ROUTINE_PERTURB('pert18_vs340','lowres_spinup_aginglaw_reference')

fig,ax = plt.subplots(figsize=(15,6))
ax,lag = routine.pub_plot_triggering_response(ax)
# ax,lag = routine.plot_triggering_response(ax)
# ax = routine.plot_compare_with_input_dCFS(ax,dt=0.01)
plt.tight_layout()
plt.show()


# Initiate

In [1]:
import numpy as np
from perturb_tools import PERTURB
from perturb_plots import *
import matplotlib.pylab as plt
import myplots
mp = myplots.Figpref()
plt.rcParams['font.size'] = '15'

ref = PERTURB('lowres_spinup_aginglaw_reference')
model1 = PERTURB('after_pert18_vs340_lowres_spinup_aginglaw_reference')
# ref = PERTURB('reference')
# model1 = PERTURB('after_pert56_vs340')
# model2 = PERTURB('pert20_vs340')
# model1 = PERTURB('pert12_vf320')
# model1 = PERTURB('pert31_ds330')

model1.load_output()
# model2.load_output()

# print(np.all(np.argsort(abs(model1.dep)) == np.arange(len(model1.dep))))
# print(np.all(np.argsort(abs(model2.dep)) == np.arange(len(model2.dep))))

Load saved data: /export/dump/jyun/perturb_stress/after_pert18_vs340_lowres_spinup_aginglaw_reference/outputs.npy
Load saved data: /export/dump/jyun/perturb_stress/after_pert18_vs340_lowres_spinup_aginglaw_reference/outputs_depthinfo.npy
Load saved data: /export/dump/jyun/perturb_stress/after_pert18_vs340_lowres_spinup_aginglaw_reference/const_params.npy


In [ ]:
import numpy as np
print_on = True
save_dir = '/export/dump/jyun/perturb_stress/after_pert18_vs340_lowres_spinup_aginglaw_reference'
if print_on: print('Load saved data: %s/outputs.npy'%(save_dir))
outputs = np.load('%s/outputs.npy'%(save_dir))
if print_on: print('Load saved data: %s/outputs_depthinfo.npy'%(save_dir))
dep = np.load('%s/outputs_depthinfo.npy'%(save_dir))

print(outputs.shape,dep.shape)

In [ ]:
new_outputs = np.vstack((outputs[:222],outputs[223:]))
# outputs.shape,new_outputs.shape
new_dep = np.hstack((dep[:222],dep[223:]))
print(new_outputs.shape,new_dep.shape)
print('Save data...',end=' ')
np.save('%s/outputs'%(save_dir),new_outputs)
np.save('%s/outputs_depthinfo'%(save_dir),new_dep)
print('done!')

In [ ]:
plt.rcParams['font.size'] = '15'
fig,ax=plt.subplots(figsize=(10,6))
if not 'idx' in dir(model1): model1.load_related_ref()
if not 'idx' in dir(model2): model2.load_related_ref()
tdep,tvar = model1.evdep[model1.idx],'shearT'
time,var,xlab,ylab,fign,indx = get_var(model1.outputs[:,1:,:],model1.dep,tdep,tvar,plot_in_sec=True,abs_on=True)
ax.plot(time-time[0],var-var[0],color='k',lw=2,label='pert18_vs340_lowres_spinup_aginglaw_reference')
# fout_time(ax,model1.outputs[:,1:,:],model1.dep,tdep,tvar,col='k',t0=model1.variables['time'].data[0],plot_in_sec=True,abs_on=True,lab='pert18_vs340_lowres_spinup_aginglaw_reference')
tdep = model2.evdep[model2.idx]
time,var,xlab,ylab,fign,indx = get_var(model2.outputs[:,1:,:],model2.dep,tdep,tvar,plot_in_sec=True,abs_on=True)
ax.plot(time-time[0],var-var[0],color=mp.mypink,lw=2,label='pert20_vs340')
# fout_time(ax,model2.outputs[:,1:,:],model2.dep,tdep,tvar,col=mp.mypink,t0=model2.variables['time'].data[0],plot_in_sec=True,abs_on=True,lab='pert20_vs340')
ax.legend(fontsize=13)
ax.set_xlabel('Time [s]',fontsize=17)
ax.set_ylabel('Shear Traction [MPa]',fontsize=17)
plt.show()


# Define plotting tools

In [ ]:
import matplotlib.pylab as plt
from scipy import signal,interpolate
from perturb_plots import *
import myplots
mp = myplots.Figpref()
plt.rcParams['font.size'] = '15'

def along_fault_at_transition(save_dir,outputs,dep,target_var,ls='-',col='k',lab='',print_on=True,save_on=True):
    if print_on:
        print('At time %1.4f s'%(outputs[0,0]))
    if target_var == 'state':
        var = np.array(outputs[:,1])
        ylab = 'State Variable'
        fign = 'state'
    elif target_var == 'slip':
        var = np.array(outputs[:,2])
        ylab = 'Cumulative Slip [m]'
        fign = 'cumslip'
    elif target_var == 'shearT':
        var = abs(np.array(outputs[:,3]))
        ylab = 'Absolute Shear Stress [MPa]'
        fign = 'shearT'
    elif target_var == 'sliprate':
        if np.all(np.array(outputs[:,4])>0):
            var = np.log10(np.array(outputs[:,4]))
        else:
            print('Negative slip rate - taking absolute')
            var = np.log10(abs(np.array(outputs[:,4])))
        ylab = 'log$_{10}$(Slip Rate [m/s])'
        fign = 'sliprate'
    elif target_var == 'normalT':
        var = abs(np.array(outputs[:,5]))
        ylab = 'Absolute Normal Stress [MPa]'
        fign = 'normalT'

    plt.plot(var,-dep,color=col, lw=2.5,label=lab,linestyle=ls)
    plt.xlabel(ylab,fontsize=17)
    plt.ylabel('Depth [km]',fontsize=17)
    plt.ylim(min(abs(dep)),max(abs(dep)))    
    plt.tight_layout()
    if save_on:
        plt.savefig('%s/%s_at_transition.png'%(save_dir,fign))

def get_lag(base_t,var_ref,t_ref,var_pert,t_pert,print_on=False):
    dat_ref = interpolate.interp1d(t_ref,var_ref)(base_t) # reference model 
    if base_t.max()>t_pert.max() or base_t.min()<t_pert.max():
        dat_pert = interpolate.interp1d(t_pert,var_pert)(base_t[np.logical_and(base_t<=t_pert.max(),base_t>=t_pert.min())])
    else:
        dat_pert = interpolate.interp1d(t_pert,var_pert)(base_t)
    dat_ref -= np.mean(dat_ref)
    dat_pert -= np.mean(dat_pert)
    corr = signal.correlate(dat_ref,dat_pert)
    lags = signal.correlation_lags(len(dat_ref),len(dat_pert))
    dat_corr = corr/max(abs(corr))
    dt = np.diff(base_t)[0]
    lag = lags[np.argmax(dat_corr)]*dt
    if print_on:
        if lag > 0:
            print('Time advance of %1.4f s'%lag)
        elif lag == 0:
            print('No change in event time')
        else:
            print('Time delay of %1.4f s'%lag)
    return lag

def stress_pert_at_depth(save_dir,ref_outputs,dep,delVar,depth_range,target_depth,target_var,plot_in_sec,dt=0.01,ls='-',col='k',lab='',abs_on=False,print_on=True,save_on=True):
    indx = np.argmin(abs(abs(dep) - abs(target_depth)))
    print('Depth = %1.1f [km]'%abs(dep[indx]))
    t0 = ref_outputs[0,0,0]
    time = np.linspace(t0,t0+delVar.shape[0]*dt,delVar.shape[0])
    di = np.argmin(abs(depth_range+target_depth))
    if target_var == 'shearT':
        if abs_on:
            var = abs(np.array(ref_outputs[indx])[:,3])
            ylab = 'Absolute Shear Stress [MPa]'
        else:
            var = np.array(ref_outputs[indx])[:,3]
            ylab = 'Shear Stress [MPa]'
        fign = 'shearT'
    elif target_var == 'normalT':
        if abs_on:
            var = abs(np.array(ref_outputs[indx])[:,5])
            ylab = 'Absolute Normal Stress [MPa]'
        else:
            var = np.array(ref_outputs[indx])[:,5]
            ylab = 'Normal Stress [MPa]'
        fign = 'normalT'
    if plot_in_sec:        # --- Plot in seconds
        plt.plot(time,delVar[:,di]+var[0],color=col,lw=2.5,label=lab,linestyle=ls)
        # plt.plot(time,delVar[:,di]+var[1],color=col,lw=2.5,label=lab,linestyle=ls)
        plt.xlabel('Time [s]',fontsize=17)
        otime = time
    else:        # --- Plot in years
        plt.plot(time/sc.yr2sec,delVar[:,di]+var[0],color=col, lw=2.5,label=lab,linestyle=ls)
        # plt.plot(time/sc.yr2sec,delVar[:,di]+var[1],color=col, lw=2.5,label=lab,linestyle=ls)
        plt.xlabel('Time [yrs]',fontsize=17)
        otime = time/sc.yr2sec
    plt.ylabel(ylab,fontsize=17)
    plt.tight_layout()
    if save_on:
        plt.savefig('%s/%s.png'%(save_dir,fign))
    return otime,delVar[:,di]+var[1]

# Peak value along the whole fault

In [ ]:
plt.rcParams['font.size'] = '15'
plt.figure(figsize=(15,6))
tvar = 'sliprate'
fout_time_max(save_dir2,outputs2[:,1:,:],tvar,lab='During perturbation',plot_in_sec=True,col=mp.mypink,save_on=False)
fout_time_max(save_dir3,outputs3[:,1:,:],tvar,lab='After perturbation',plot_in_sec=True,col=mp.myblue,save_on=False)
yl = plt.gca().get_ylim()
plt.vlines(x=tstart[idx],ymin=yl[0],ymax=yl[1],color=mp.mynavy,linestyles='--')
plt.ylim(yl)
plt.legend(fontsize=15,loc='upper left')
plt.show()

In [ ]:
plt.rcParams['font.size'] = '15'
plt.figure(figsize=(15,6))
tvar = 'sliprate'
inc = 1e3
if 'save_dir3' not in locals(): 
    i1 = np.where(np.logical_and(outputs1[0,:,0]>=outputs2[0,0,0]-inc,outputs1[0,:,0]<=outputs2[0,-1,0]+inc))[0]
else:
    i1 = np.where(np.logical_and(outputs1[0,:,0]>=outputs2[0,0,0]-inc,outputs1[0,:,0]<=outputs3[0,-1,0]+inc))[0]
# fout_time_max(save_dir1,outputs1[:,i1,:],tvar,lab='Reference (hf = 25 m)',plot_in_sec=False,save_on=False)
fout_time_max(save_dir2,outputs2,tvar,lab='Reference (hf = 10 m)',plot_in_sec=False,col=mp.mypink,save_on=False)
add = (outputs2[0,-1,0]-outputs2[0,0,0])*0.2
yl = plt.gca().get_ylim()
plt.ylim(yl)
plt.legend(fontsize=15,loc='upper left')
plt.show()

# Value at certain depth, within specific time range

In [ ]:
plt.rcParams['font.size'] = '15'
fig,ax=plt.subplots(figsize=(10,6))
if not 'idx' in dir(model1):
    model1.load_related_ref()
tdep,tvar = model1.evdep[model1.idx],'slip'
fout_time(ax,model1.outputs[:,1:,:],model1.dep,tdep,tvar,col='k',plot_in_sec=True,lab='Stress-dependent aging law')
# plt.legend(fontsize=15,loc='lower right')
plt.show()


In [ ]:
np.all(np.diff(cumslip) == np.zeros(len(np.diff(cumslip))))

In [ ]:
fig,ax=plt.subplots(figsize=(10,6))
di = np.argmin(abs(abs(model1.dep) - abs(tdep)))
time = model1.variables['time'].data
# cumslip = model1.variables['slip'].data[di,:]
slip = cumslip - cumslip[0]
# plt.plot(time,cumslip,'k')
ds = np.diff(cumslip)
dt = np.diff(time)
plt.plot(time[1:],ds/dt,'k')
# plt.plot(time,slip,'k')
plt.xlabel('Time [s]',fontsize=17)
# plt.xlabel('Time [s]',fontsize=17)
plt.show()

In [ ]:
plt.rcParams['font.size'] = '15'
plt.figure(figsize=(10,6))
# plt.figure(figsize=(15,6))
tdep,tvar = evdep[idx],'state'
# i3 = np.where(outputs3[0,:,0]<=outputs3[0,0,0]+100)[0]
i1 = np.where(np.logical_and(outputs1[0,:,0]<=outputs2[0,-1,0]+1e4,outputs1[0,:,0]>=outputs2[0,0,0]-1e4))[0]
# fout_time(save_dir2,outputs2,dep2,tdep,tvar,col=mp.mypink,plot_in_sec=True,lab='During perturbation',save_on=False)
# fout_time(save_dir3,outputs3[:,i3,:],dep3,tdep,tvar,col=mp.myblue,plot_in_sec=True,lab='After perturbation',save_on=False)
fout_time(save_dir2,outputs2[:,1:,:],dep2,tdep,tvar,col=mp.mypink,plot_in_sec=True,lab='Stress-dependent aging law',save_on=False)
# fout_time(save_dir4,outputs4[:,1:,:],dep4,tdep,tvar,col='k',plot_in_sec=True,lab='Basic aging law',save_on=False,ls='--')
fout_time(save_dir1,outputs1[:,i1,:],dep1,tdep,tvar,col='k',plot_in_sec=True,lab='Basic aging law',save_on=False,ls='--')
plt.legend(fontsize=15,loc='lower right')
# plt.legend(fontsize=15,loc='upper right')
plt.xlim(outputs2[0,0,0]-.5,outputs2[0,-1,0]+.5)
# plt.xlim(outputs2[0,0,0]-.5,outputs3[0,0,0]+5)
# plt.xlim(outputs2[0,-1,0]-1.5,outputs3[0,0,0]+1.5)
plt.show()


In [ ]:
def compute_RHS(outputs,dep,tdep):
    indx = np.argmin(abs(abs(dep) - abs(tdep)))
    print('Depth = %1.2f [km]'%abs(dep[indx]))
    V0 = 1e-6
    time = outputs[indx,1:,0]-outputs[indx,1,0]
    psi = outputs[indx,1:,1]
    V = outputs[indx,1:,4]
    sn = outputs[indx,1:,5]
    mesh_y,het_var = np.loadtxt('/home/jyun/Tandem/perturb_stress/fractal_ab_02',unpack=True)
    from scipy import interpolate
    f_var = interpolate.interp1d(mesh_y,het_var,bounds_error=False,fill_value=het_var[0])
    a = f_var(dep)[indx] + 0.019
    f = a*np.arcsinh((abs(V)/2/V0) * np.exp(psi/a))
    tau = -sn*f
    return time,psi,tau,V,sn,a

In [ ]:
plt.rcParams['font.size'] = '15'
plt.figure(figsize=(15,6))
tdep,tvar = evdep[idx],'state'
inc = 1e4
i1 = np.where(np.logical_and(outputs1[0,:,0]>=tstart[idx]-58320,outputs1[0,:,0]<=tstart[idx]))[0]
i3 = np.where(np.logical_and(outputs3[0,:,0]>=tstart[idx]-58320,outputs3[0,:,0]<=tstart[idx]))[0]
fout_time(save_dir1,outputs1[:,i1,:],dep1,tdep,tvar,plot_in_sec=True,lab='Unperturbed',save_on=False)
fout_time(save_dir2,outputs2,dep2,tdep,tvar,col=mp.mypink,plot_in_sec=True,lab='During perturbation',save_on=False)
fout_time(save_dir3,outputs3[:,i3,:],dep3,tdep,tvar,col=mp.myblue,plot_in_sec=True,lab='After perturbation',save_on=False)
plt.legend(fontsize=15,loc='lower right')
# plt.legend(fontsize=15,loc='upper right')
# plt.xlim(outputs2[0,0,0]-1.5,outputs3[0,0,0]+50)
plt.show()


In [ ]:
plt.rcParams['font.size'] = '15'
plt.figure(figsize=(10,6))
tdep,tvar = 22,'sliprate'
fout_time(save_dir2,outputs2[:,1:,:],dep2,tdep,tvar,col=mp.mypink,plot_in_sec=True,lab=r'$V_{pl}=3.2\times10^{-11}$ m/s',save_on=False)
fout_time(save_dir1,outputs1,dep1,tdep,tvar,col='k',plot_in_sec=True,lab='$V_{pl} = 10^{-9}$ m/s',save_on=False,ls='--')
plt.xlim(0,sc.yr2sec*2000)
plt.legend(fontsize=15)
plt.show()

# Compare slip/sliprate during perturbation

In [ ]:
target_var = 'slip'
fig,ax=plt.subplots(figsize=(6,8))
var1 = model1.variables[target_var].data
var2 = model2.variables[target_var].data
print(model1.variables['time'].data[0,-1]-model1.variables['time'].data[0,0])
ax.plot(var1[:,-1]-var1[:,0],-model1.dep,lw=2.5,label=model1.branch_name)
ax.plot(var2[:,-1]-var2[:,0],-model2.dep,lw=2.5,label=model2.branch_name)
ax.legend(fontsize=14)
ax.set_xlabel(model1.variables[target_var].label,fontsize=17)
ax.set_ylabel('Depth [km]',fontsize=17)
ax.grid(True,alpha=0.5)
ax.set_ylim(0,15)
ax.invert_yaxis()

# Check the last written output

In [ ]:
fig,ax=plt.subplots(ncols=2,figsize=(15,10))
ivar = 3
# ax[0].plot(outputs2[:,-1,ivar],-dep2,c='k',lw=2.5,label='Last output during perturbation')
# ax[0].plot(outputs3[:,1,ivar],-dep3,c=mp.myburgundy,lw=2.5,linestyle='--',label='First output after perturbation')
# ax[0].legend(fontsize=14)
ax[0].plot(outputs3[:,1,ivar]-outputs2[:,-1,ivar],-dep2,c='k',lw=2.5,label='Last output during perturbation')
ax[0].set_xlabel('Shear Stress [MPa]',fontsize=17)
ax[0].set_ylabel('Depth [km]',fontsize=17)
ax[0].grid(True,alpha=0.5)
ax[0].set_ylim(0,24)
ax[0].invert_yaxis()

ivar = 5
# ax[1].plot(outputs2[:,-1,ivar],-dep2,c='k',lw=2.5,label='Last output during perturbation')
# ax[1].plot(outputs3[:,1,ivar],-dep3,c=mp.myburgundy,lw=2.5,linestyle='--',label='First output after perturbation')
# ax[1].legend(fontsize=14)
ax[1].plot(outputs3[:,1,ivar]-outputs2[:,-1,ivar],-dep2,c='k',lw=2.5,label='Last output during perturbation')
ax[1].set_xlabel('Normal Stress [MPa]',fontsize=17)
ax[1].set_ylabel('Depth [km]',fontsize=17)
ax[1].grid(True,alpha=0.5)
ax[1].set_ylim(0,24)
ax[1].invert_yaxis()
plt.show()

In [ ]:
plt.rcParams['font.size'] = '15'
plt.figure(figsize=(6,8))
target_var = 'state'
along_fault_at_transition(save_dir1,outputs1[:, outputs1.shape[1]-1,:],dep1,target_var,col='0.5',lab='output1; End',save_on=False)
along_fault_at_transition(save_dir2,outputs2[:,0,:],dep2,target_var,col=mp.mypink,lab='output2; Start',save_on=False)
along_fault_at_transition(save_dir2,outputs2[:, outputs2.shape[1]-1,:],dep2,target_var,col='0.62',lab='output2; End',save_on=False)
along_fault_at_transition(save_dir3,outputs3[:,0,:],dep3,target_var,col=mp.myblue,lab='output3; Start',save_on=False)
plt.legend(fontsize=15,loc='lower left')
plt.gca().invert_yaxis()

In [ ]:
plt.rcParams['font.size'] = '15'
plt.figure(figsize=(6,8))
target_var = 'state'
indx1,indx2 = outputs1.shape[1]-1, 0
along_fault_at_transition(save_dir2,outputs2[:,0,:]-outputs1[:,outputs1.shape[1]-1,:],dep2,target_var,lab='output2 - output1',col=mp.myburgundy,save_on=False)
along_fault_at_transition(save_dir3,outputs3[:,0,:]-outputs2[:,outputs2.shape[1]-1,:],dep3,target_var,lab='output3 - output2',col=mp.mynavy,save_on=False)
plt.legend(fontsize=15,loc='lower left')
plt.gca().invert_yaxis()

In [ ]:
print('Outputs1: from %1.18e to %1.18e s'%(outputs1[0,0,0],outputs1[0,-1,0]))
print('Time at checkpoint 1: %1.18e s'%(ckp_dat1[-1][-1]))
print('Outputs2: from %1.18e to %1.18e s'%(outputs2[0,0,0],outputs2[0,-1,0]))
print('Time at checkpoint 2: %1.18e s'%(ckp_dat2[-1][-1]))
print('Outputs3: from %1.18e to %1.18e s'%(outputs3[0,0,0],outputs3[0,-1,0]))

# Plot τ/σ

In [ ]:
target_depth = evdep[idx]
indx = np.argmin(abs(abs(dep2) - abs(target_depth)))
print('Depth: %1.2f km'%(-dep2[indx]))
pure_Ts = np.array(outputs2[indx])[:,3]
pure_Pn = np.array(outputs2[indx])[:,5]
rel_time = np.array(outputs2[indx])[:,0] - np.array(outputs2[indx])[0,0]

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(rel_time,-pure_Ts/pure_Pn,'k',lw=2.5)
plt.xlabel('Time [s]',fontsize=17)
plt.ylabel(r'$\tau/\sigma$',fontsize=17)
plt.grid(True,alpha=0.5)
plt.ylim(0.3,0.8)
plt.show()

# Work

In [ ]:
from scipy import integrate
shearT = abs(model1.variables['shearT'].data)
cumslip = model1.variables['slip'].data[:,1:]

G,Dev = [],[]
for di in range(len(model1.dep)):
    target_depth = model1.dep[di]
    slip = cumslip[di,:] - cumslip[di,0]
    ts = -model1.get_output_dCFS(target_depth,print_on=False)[-1]

    x = slip[:400]
    y = ts[:400]*1e6
    area = integrate.simpson(y,x)
    G.append(area)
    Dev.append(x[-1]-x[0])

work = integrate.simpson(G,-model1.dep*1e3)
# print(work)

In [ ]:
if not 'idx' in dir(model1):
    model1.load_related_ref()
    hypo_depth = model1.evdep[model1.idx]
slip = cumslip[:,-1] - cumslip[:,0]
fig,ax=plt.subplots(ncols=2,figsize=(10,6))
ax[0].plot(slip,-model1.dep,'k')
xl = ax[0].get_xlim()
ax[0].hlines(y=hypo_depth,xmin=xl[0],xmax=xl[1],linestyle='--',color=mp.myburgundy)
ax[0].set_xlim(xl)
ax[0].set_xlabel('Slip [m]')
ax[0].set_ylabel('Depth [km]')
ax[0].set_ylim(0,12)
ax[0].invert_yaxis()

ax[1].plot(G,-model1.dep,'k')
xl = ax[1].get_xlim()
ax[1].hlines(y=hypo_depth,xmin=xl[0],xmax=xl[1],linestyle='--',color=mp.myburgundy)
ax[1].set_xlim(xl)
ax[1].set_xlabel('Work per distance [J/m]')
ax[1].set_ylim(0,12)
ax[1].invert_yaxis()
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(5,6))
# ax[0].plot(shearT[:,0],-model1.dep,'k')
# ax[0].plot(shearT[:,-1],-model1.dep,'r')
# ax.plot(shearT[:,-1]-shearT[:,0],-model1.dep,'k')
ax.plot(shearT[:,400]-shearT[:,0],-model1.dep,'k')
xl = ax.get_xlim()
ax.hlines(y=hypo_depth,xmin=xl[0],xmax=xl[1],linestyle='--',color=mp.myburgundy)
ax.set_xlim(xl)
ax.set_xlabel('Shear Traction [MPa]')
ax.set_ylabel('Depth [km]')
ax.set_ylim(0,24)
ax.invert_yaxis()
plt.show()

# Compare with the input stress perturbation

In [ ]:
delPn = np.loadtxt('%s/ssaf_%s_Pn_pert_mu%02d_%d.dat'%(pert.stress_models_dir,pert.seissol_model_n,int(pert.mu*10),pert.receivef_strike))
delTs = np.loadtxt('%s/ssaf_%s_Ts_pert_mu%02d_%d.dat'%(pert.stress_models_dir,pert.seissol_model_n,int(pert.mu*10),pert.receivef_strike))
depth_range = np.loadtxt('%s/ssaf_%s_dep_stress_pert_mu%02d_%d.dat'%(pert.stress_models_dir,pert.seissol_model_n,int(pert.mu*10),pert.receivef_strike))

In [ ]:
if not 'idx' in dir(model1):
    model1.load_related_ref()
    target_depth = model1.evdep[model1.idx]

## Compare with input normal stress/shear stress 

### Stress change

In [ ]:
from scipy import interpolate
# target_depth = 6.9
# target_depth = -model1.dep[np.argmax(cumslip[:,-1]-cumslip[:,0])]
if not 'idx' in dir(model1):
    model1.load_related_ref()
target_depth = model1.evdep[model1.idx]
_,time_seissol,delPn,delTs,depth_range = model1.get_input_dCFS(target_depth,dt=0.01)
_,time_tandem,pn,ts = model1.get_output_dCFS(target_depth)
delPn_at_D = np.array([interpolate.interp1d(depth_range,delPn[ti])(-target_depth) for ti in range(delPn.shape[0])])
delTs_at_D = [interpolate.interp1d(depth_range,delTs[ti])(-target_depth) for ti in range(delTs.shape[0])]
# t = np.linspace(0,delTs.shape[0]*0.01,delTs.shape[0])
aa=1
plt.rcParams['font.size'] = '15'
plt.figure(figsize=(10,6))
plt.plot(time_seissol,delTs_at_D,'k',lw=2.5,label='Input Shear Stress')
plt.plot(time_tandem,-ts,'0.62',lw=2.5,linestyle='--',label='Output Shear Stress')
# plt.plot(time_seissol,-delPn_at_D,c=mp.myburgundy,lw=2.5,label='Input Normal Stress')
# plt.plot(time_tandem,pn,c=mp.mypalepink,lw=2.5,linestyle='--',label='Output Normal Stress')
plt.legend(fontsize=15,loc='lower right')
plt.title('%s; Depth = %1.2f km; Strike = %d˚'%(model1.seissol_model_n,target_depth,model1.receivef_strike),fontsize=21,fontweight='bold')
plt.xlabel('Time [s]',fontsize=17)
plt.ylabel('Stress Change [MPa]',fontsize=17)
plt.grid(True,alpha=0.5)
# plt.xlim(0,6)
plt.show()

### Absolute stress level

In [ ]:
plt.rcParams['font.size'] = '15'
plt.figure(figsize=(15,6))
# tdep,tvar = evdep[idx],'normalT'
tdep,tvar = evdep[idx],'shearT'
if tvar == 'normalT':
    delVar = delPn
elif tvar == 'shearT':
    delVar = delTs
# abs_on = False
abs_on = True
inc = 1e4
i1 = np.where(np.logical_and(outputs1[0,:,0]>=outputs2[0,0,0]-inc,outputs1[0,:,0]<=outputs2[0,-1,0]+inc))[0]
fout_time(save_dir1,outputs1[:,i1,:],dep1,tdep,tvar,plot_in_sec=True,col='0.62',lab='Unperturbed',abs_on=abs_on,save_on=False)
otime,_ = stress_pert_at_depth(save_dir2,outputs2,dep2,delVar,depth_range,tdep,tvar,plot_in_sec=True,col='k',lab='Input perturbation',abs_on=abs_on,print_on=True,save_on=False)
fout_time(save_dir2,outputs2,dep2,tdep,tvar,col=mp.mypink,plot_in_sec=True,lab='Perturbed',abs_on=abs_on,save_on=False)
plt.legend(fontsize=15,loc='lower right')
plt.xlim(otime[0]-5,otime[-1]+5)
# plt.xlim(outputs2[0,0,0]-2.5,outputs2[0,-1,0]+2.5)
# plt.ylim(-36.9,-35)
# plt.ylim(35.25,36.5)
plt.show()


## Compute dCFS from the output to validate the behvaior

### During perturbation vs. input

In [ ]:
from perturb_plots import compare_dCFS_at_depth
plt.rcParams['font.size'] = '15'
fig,ax=plt.subplots(figsize=(15,6))
if not 'idx' in dir(model1):
    model1.load_related_ref()
    target_depth = model1.evdep[model1.idx]
dCFSt_seissol,time_seissol = model1.get_input_dCFS(target_depth,dt=0.01)[:2]
dCFSt_tandem,time_tandem = model1.get_output_dCFS(target_depth)[:2]
ax = compare_dCFS_at_depth(ax,dCFSt_seissol,time_seissol,dCFSt_tandem,time_tandem)
ax.legend(fontsize=15,loc='lower right')
plt.show()

### After perturbation vs. During perturbation

In [ ]:
from scipy import interpolate
tdep = evdep[idx]
mu = 0.4
inc = 1e4

indx = np.argmin(abs(abs(dep2) - abs(tdep)))
print('Depth = %1.2f [km]'%abs(dep2[indx]))
pn2 = np.array(outputs2[indx])[1:,5]-np.array(outputs2[indx])[1,5]
ts2 = np.array(outputs2[indx])[1:,3]-np.array(outputs2[indx])[1,3]
dCFSt2 = -ts2 - mu*pn2
t2 = np.array(outputs2[indx])[1:,0] # -np.array(outputs2[indx])[0,0]

pn3 = np.array(outputs3[indx])[1:,5]-np.array(outputs2[indx])[1,5]
ts3 = np.array(outputs3[indx])[1:,3]-np.array(outputs2[indx])[1,3]
dCFSt3 = -ts3 - mu*pn3
t3 = np.array(outputs3[indx])[1:,0] # -np.array(outputs3[indx])[0,0]

plt.rcParams['font.size'] = '15'
plt.figure(figsize=(15,6))

plt.plot(t2,dCFSt2,color='k',lw=2.5,label='DuringPerturbation',linestyle='-')
plt.plot(t3,dCFSt3,color=mp.myburgundy,lw=2.5,label='After Perturbation',linestyle='--')
plt.xlabel('Time [s]',fontsize=17)
plt.ylabel('dCFS [Mpa]',fontsize=17)
plt.grid(True,alpha=0.5)
plt.tight_layout()

plt.legend(fontsize=15,loc='lower right')
plt.xlim(outputs2[0,0,0]-.5,outputs3[0,0,0]+5)
plt.ylim(-1.5,1.5)
plt.show()


# Compare event distribution

In [ ]:
def plot_system_events(tstart,evdep,system_wide,col,lab=''):
    # plt.scatter(np.arange(len(system_wide)),evdep[system_wide],150,ec='k',fc=col,lw=1,marker='*',zorder=3,label=lab)
    plt.scatter(tstart[system_wide],evdep[system_wide],150,ec='k',fc=col,lw=1,marker='*',zorder=3,label=lab)
    xl = ax.get_xlim()
    plt.hlines(y=2,xmin=xl[0],xmax=xl[1],linestyles='--',color='0.62',lw=1.5)
    plt.hlines(y=12,xmin=xl[0],xmax=xl[1],linestyles='--',color='0.62',lw=1.5)
    plt.ylim(0,24)
    ax.invert_yaxis()
    plt.xticks(np.arange(len(system_wide)), minor=True)
    # plt.xlabel('System-size event index',fontsize=17)
    plt.xlabel('Event Time [s]',fontsize=17)
    plt.ylabel('Depth [km]',fontsize=17)
    plt.xlim(xl[0],xl[1])
    plt.grid(True,alpha=0.5,which='both')

def plot_all_events(tstart,evdep,system_wide,partial_rupture,lead_fs):
    plt.scatter(tstart[partial_rupture],evdep[partial_rupture],70,ec='k',fc=mp.mylightblue,lw=1,marker='d',zorder=3,label='Partial rupture events')
    plt.scatter(tstart[lead_fs],evdep[lead_fs],70,ec='k',fc=mp.mydarkviolet,lw=1,marker='d',zorder=3,label='Leading foreshocks')
    plt.scatter(tstart[system_wide],evdep[system_wide],150,ec='k',fc=mp.mydarkviolet,lw=1,marker='*',zorder=3,label='System-size events')
    # for k in range(len(tstart)):
    #     plt.text(tstart[k],evdep[k]-0.2,'%d'%(k),color='k',fontsize=13,ha='right',va='bottom')
    xl = ax.get_xlim()
    plt.hlines(y=2,xmin=0,xmax=xl[1],linestyles='--',color='0.62',lw=1.5)
    plt.hlines(y=12,xmin=0,xmax=xl[1],linestyles='--',color='0.62',lw=1.5)
    plt.ylim(0,24)
    ax.invert_yaxis()
    plt.xlabel('Event Time [s]',fontsize=17)
    plt.ylabel('Depth [km]',fontsize=17)
    plt.xlim(0,xl[1])
    # plt.xlim(xl)
    plt.grid(True,alpha=0.5,which='both')

In [ ]:
# System-size events only
ii = np.where(tstart[system_wide] <= max(tstart2))[0]
plot_system_events(tstart,evdep,system_wide[ii],mp.mydarkviolet)
plot_system_events(tstart2,evdep2,system_wide2,mp.myyellow,lab='$V_{pl} = 1$ mm/yr')
print('Vpl = 31 mm/yr: last event: %g s, total %d events'%(np.max(tstart[system_wide[ii]]),len(system_wide[ii])))
print('Vpl = 1 mm/yr: last event: %g s, total %d events'%(np.max(tstart2[system_wide2[ii]]),len(system_wide2)))
plt.legend(fontsize=13,loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
# ref.load_events()
# model1.load_events(compute_on=True,save_on=False)
fig,ax=plt.subplots(figsize=(18,6))
plot_all_events(ref.tstart,ref.evdep,ref.system_wide,ref.partial_rupture,ref.lead_fs)
plt.vlines(x=model1.tstart[model1.system_wide],ymin=0,ymax=24,color=mp.myburgundy,linestyles='--',lw=2)
plt.legend(fontsize=13,loc='lower right')
plt.tight_layout()
plt.show()

In [2]:
ref.load_events()
model1.load_events(compute_on=True,save_on=False)
ref.get_idx(model1.target_eventid)
# fig,ax=plt.subplots(figsize=(8,6))
# plot_all_events(ref.tstart,ref.evdep,ref.system_wide,ref.partial_rupture,ref.lead_fs)
# plt.vlines(x=ref.tstart[ref.idx]-16.2*3600,ymin=0,ymax=24,color=mp.mypink,linestyles='--',lw=2)
# plt.text(ref.tstart[ref.idx]-16.2*3600-1e4,15,'Perturbation Start',color=mp.mypink,fontsize=13,ha='right',va='bottom',fontweight='bold')
# plt.vlines(x=model1.tstart[model1.system_wide],ymin=0,ymax=24,color=mp.mynavy,linestyles='--',lw=2)
# plt.text(model1.tstart[model1.system_wide]-1e4,13.5,'Perturbed Event Time',color=mp.mynavy,fontsize=13,ha='right',va='bottom',fontweight='bold')
# # xmin,xmax = ref.tstart[12]-1e5,model1.tend[model1.system_wide[-1]]+1e5
# closest_leadfs = ref.lead_fs[np.argmin(ref.tstart[ref.lead_fs]-ref.tstart[ref.idx]-16.2*3600)]
# print(closest_leadfs)
# xmin,xmax = ref.tstart[closest_leadfs]-1e5,model.tend[model.system_wide[-1]]+1e5
# for k in range(len(ref.tstart)):
#     if ref.tstart[k] >= xmin and ref.tstart[k] <= xmax:
#         plt.text(ref.tstart[k],ref.evdep[k]-0.2,'%d'%(k),color='k',fontsize=13,ha='right',va='bottom')
# plt.legend(fontsize=13,loc='lower right')
# plt.xlim(xmin,xmax)
# plt.tight_layout()
# plt.show()

Load saved data: /export/dump/jyun/perturb_stress/lowres_spinup_aginglaw_reference/cumslip_outputs_Vths_2e-01_srvar_010_rths_10_tcreep_2_tseis_05.npy
Load saved data: /export/dump/jyun/perturb_stress/lowres_spinup_aginglaw_reference/spin_up_idx_Vths_2e-01_srvar_010_rths_10_tcreep_2_tseis_05.npy
Compute event details
Compute cumulative slip vs. depth >>> No cutting
Slip rate > 2e-01
All safe from the SR variation criterion
Remove single-depth activated event: []
Too short to spin-up - skip


In [4]:
(ref.tstart[18]-ref.tstart[12])/3600

116.20299906571707

In [ ]:
ref.lead_fs[np.argmin(abs(ref.tstart[ref.lead_fs]-ref.tstart[ref.idx]-16.2*3600))]
# np.argmin(abs(ref.tstart[ref.lead_fs]-ref.tstart[ref.idx]-16.2*3600))

# Quick summary chart

## Fixed event

## Fixed stress perturbation model

In [ ]:
from perturb_tools import PERTURB
import setup_shortcut
sc = setup_shortcut.setups()
import numpy as np

print('####### Fixed stress perturbation model #######')
model_n = 'vert_slow'
receivef_strike = 340
time_diff_in_h = 30
if time_diff_in_h == 16.2:
    full_model_name = '%s%d'%(sc.model_code(model_n),receivef_strike)
else:
    full_model_name = '%s%d_%dh'%(sc.model_code(model_n),receivef_strike,time_diff_in_h)
print('Model: %s'%(full_model_name))

event_list = [8,12,20,31,45,56]

evidx,org_evdep,new_evdep,trigg_response,peak_dyn,static = [],[],[],[],[],[]
for target_eventid in event_list:
    pert = PERTURB(target_eventid,model_n,receivef_strike,print_on=False)
    pert.events_without_pert(print_on=False)
    pert.events_with_pert(print_on=False)
    pert.load_stress_info(print_on=False)
    org_evdep.append(pert.ref_evdep[pert.ref_idx])
    evidx.append(pert.ref_idx)
    if len(pert.system_wide) == 1:
        new_evdep.append(pert.evdep[pert.system_wide][0])
        trigg_response.append((pert.tstart[pert.system_wide]-pert.ref_tstart[pert.ref_idx])/3600)
        peak_dyn.append(pert.peak_dynamic)
        static.append(pert.static)
    elif len(pert.system_wide) > 1:
        print('Event %d: More than one system-size events'%(target_eventid))
    elif len(pert.system_wide) == 0:
        print('Event %d: No system-size events'%(target_eventid))

print(' Event # | ID | Hyp. Depth | Peak dCFS | Static dCFS | Time diff. | New Hyp. Depth')
for i in range(len(org_evdep)):
    print('  %d  |  %d  |   %1.2f   |   %1.4f   |   %1.4f   |   %1.4f  |   %1.2f  '%(evidx[i],event_list[i],org_evdep[i],peak_dyn[i],static[i],trigg_response[i],new_evdep[i]))

In [ ]:
from perturb_tools import PERTURB
import setup_shortcut
sc = setup_shortcut.setups()
import numpy as np

print('####### Fixed stress perturbation model #######')
model_n = 'vert_slow'
receivef_strike = 340
time_diff_in_h = 30
if time_diff_in_h == 16.2:
    full_model_name = '%s%d'%(sc.model_code(model_n),receivef_strike)
else:
    full_model_name = '%s%d_%dh'%(sc.model_code(model_n),receivef_strike,time_diff_in_h)
print('Model: %s'%(full_model_name))

pert = PERTURB(8,model_n,receivef_strike,print_on=False)
pert.events_without_pert(print_on=False)
pert.load_stress_info(print_on=False)
print(pert.peak_dynamic,pert.static)

In [ ]:
float('vert_slow_X10'.split('_X')[-1])